In [1]:
!pip install evaluate rouge_score absl-py

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 44.1 MB/s eta 0:00:00
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=dbddb

In [2]:
# Argilla 뉴스 요약 데이터셋 로드, 전처리 및 데이터 분할
import numpy as np # 넘파이 라이브러리 임포트
from datasets import load_dataset # Hugging Face의 datasets 라이브러리에서 load_dataset 함수 임포트

# 'argilla/news-summary' 데이터셋을 불러와 'test' 분할을 선택합니다.
# news = load_dataset(...) : 데이터셋을 news 변수에 할당합니다.
news = load_dataset("argilla/news-summary", split="test")

# 데이터셋을 판다스 데이터프레임으로 변환하고, 5000개 샘플을 무작위로 추출합니다.
# df = news.to_pandas().sample(...) : 추출된 샘플을 df 변수에 할당합니다.
df = news.to_pandas().sample(5000, random_state=42)[["text", "prediction"]]

# 'prediction' 컬럼의 데이터를 첫 번째 텍스트 값으로 매핑합니다.
# .map(lambda x: x[0]["text"]) : 각 prediction 항목이 리스트 내 딕셔너리 형태이므로, 첫 번째 딕셔너리의 'text' 값을 추출합니다.
df["prediction"] = df["prediction"].map(lambda x: x[0]["text"])

# 전체 데이터를 6:2:2 비율로 훈련, 검증, 테스트 데이터셋으로 분할합니다.
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# 첫 번째 훈련 데이터의 원문(text)과 요약문(prediction)의 일부를 출력합니다.
print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")

# 분할된 각 데이터셋의 크기를 출력합니다.
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-ebc48879f34571(…):   0%|          | 0.00/1.54M [00:00<?, ?B/s]

data/test-00000-of-00001-6227bd8eb10a9b5(…):   0%|          | 0.00/31.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20417 [00:00<?, ? examples/s]

Source News : DANANG, Vietnam (Reuters) - Russian President Vladimir Putin said on Saturday he had a normal dialogue with U.S. leader Donald Trump at a summit in Vietnam, and described Trump as civil, well-educated
Summarization : Putin says had useful interaction with Trump at Vi
Training Data Size : 3000
Validation Data Size : 1000
Testing Data Size : 1000


/opt/conda/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 32.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers] [transformers]


In [5]:
# Bart 모델 학습을 위한 데이터 전처리 및 데이터로더 생성
import torch # 파이토치 라이브러리 임포트
from transformers import BartTokenizer # Hugging Face의 transformers 라이브러리에서 BartTokenizer 임포트
from torch.utils.data import TensorDataset, DataLoader # 파이토치 데이터 유틸리티 임포트
from torch.utils.data import RandomSampler, SequentialSampler # 데이터 샘플러 임포트
from torch.nn.utils.rnn import pad_sequence # 시퀀스 길이를 맞추기 위한 함수 임포트

# 데이터셋을 만드는 함수를 정의합니다.
def make_dataset(data, tokenizer, device):
    # 입력 텍스트(원문)를 토크나이징합니다.
    tokenized = tokenizer(
        text=data.text.tolist(), # 데이터프레임의 'text' 컬럼을 리스트로 변환하여 입력으로 사용
        padding="longest", # 모든 시퀀스 길이를 가장 긴 시퀀스에 맞춰 패딩
        truncation=True, # 시퀀스 길이가 최대 길이를 초과하면 잘라냄
        return_tensors="pt", # 파이토치 텐서 형태로 반환
        max_length=1024 # 최대 시퀀스 길이를 1024로 설정
    )
    labels = [] # 라벨(요약문)을 저장할 빈 리스트 생성
    input_ids = tokenized["input_ids"].to(device) # 토큰 인덱스를 추출하여 device로 이동
    attention_mask = tokenized["attention_mask"].to(device) # 어텐션 마스크를 추출하여 device로 이동
    
    # 각 요약문(prediction)을 토크나이징하고 labels 리스트에 추가합니다.
    for target in data.prediction:
        labels.append(tokenizer.encode(target, return_tensors="pt").squeeze())
        # tokenizer.encode()는 텍스트를 토큰화하여 텐서로 반환합니다. .squeeze()로 차원을 축소합니다.
    
    # 여러 길이의 라벨 텐서를 가장 긴 길이에 맞춰 패딩합니다.
    labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(device)
    # padding_value=-100: 손실 계산 시 패딩 부분을 무시하도록 -100으로 설정합니다.
    
    # input_ids, attention_mask, labels를 묶어 TensorDataset 객체를 반환합니다.
    return TensorDataset(input_ids, attention_mask, labels)

# 데이터로더를 생성하는 함수입니다.
def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset) # 데이터셋에 적용할 샘플러(Random 또는 Sequential)를 생성
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size) # DataLoader 생성
    return dataloader

# 하이퍼파라미터 및 디바이스 설정
epochs = 5
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU 사용 가능 여부 확인
tokenizer = BartTokenizer.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base") # Bart 토크나이저 로드

# make_dataset 함수를 사용하여 훈련, 검증, 테스트 데이터셋을 생성합니다.
train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)
valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)
test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

# 훈련 데이터셋의 첫 번째 요소를 출력하여 데이터 구조를 확인합니다.
print(train_dataset[0])

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

(tensor([   0,  495, 1889,  ...,    1,    1,    1], device='cuda:0'), tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0'), tensor([    0, 35891,   161,    56,  5616, 10405,    19,   140,    23,  5490,
         3564,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
       device='cuda:0'))


In [6]:
# Bart 모델 및 옵티마이저 설정
from torch import optim # 파이토치에서 옵티마이저 모듈 임포트
from transformers import BartForConditionalGeneration # Hugging Face의 transformers 라이브러리에서 BartForConditionalGeneration 클래스 임포트

# BartForConditionalGeneration 모델을 불러옵니다.
model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base").to(device)
# 'facebook/bart-base'는 사전 학습된 BART 모델의 이름입니다.
# .to(device)는 모델을 GPU와 같은 가속기로 보내 연산을 빠르게 합니다.

# AdamW 옵티마이저를 정의합니다.
optimizer = optim.AdamW(model.parameters(), lr=5e-5, eps=1e-8)
# model.parameters(): 모델의 모든 학습 가능한 파라미터를 옵티마이저에 전달합니다.
# lr=5e-5: 학습률(learning rate)을 0.00005로 설정합니다.
# eps=1e-8: 부동 소수점 연산에서 0으로 나누는 것을 방지하기 위한 작은 상수(epsilon)를 설정합니다.

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [7]:
# Bart 모델의 계층 구조 출력
# model.named_children() : 모델의 직계 자식 모듈들을 (이름, 모듈) 쌍으로 반환합니다.
for main_name, main_module in model.named_children():
    print(main_name) # 'model'과 'lm_head' 같은 최상위 모듈의 이름을 출력합니다.
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name) # 최상위 모듈의 하위 모듈 이름을 출력합니다. (예: 'model' 아래의 'shared', 'encoder', 'decoder')
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name) # 하위 모듈의 하위 모듈 이름을 출력합니다. (예: 'encoder' 아래의 'embed_tokens', 'embed_positions', 'layers')
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name) # 가장 깊은 계층의 모듈 이름을 출력합니다. (예: 'layers' 아래의 '0'부터 '11'까지)

model
└ shared
└ encoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
└ decoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
lm_head


In [8]:
# Bart 모델 훈련, 검증 및 가중치 저장
import numpy as np # 넘파이 라이브러리 임포트
import evaluate # Hugging Face의 evaluate 라이브러리 임포트
from tqdm.auto import tqdm # tqdm 라이브러리 임포트

# ROUGE 점수를 계산하는 함수입니다.
def calc_rouge(preds, labels):
    # 예측값(preds)의 가장 높은 확률을 가진 클래스의 인덱스를 가져옵니다.
    preds = preds.argmax(axis=-1)
    # 라벨에서 패딩 값(-100)을 토크나이저의 패딩 토큰 ID로 대체합니다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # 예측 토큰 ID를 텍스트로 디코딩합니다. 특수 토큰은 제외합니다.
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # 라벨 토큰 ID를 텍스트로 디코딩합니다. 특수 토큰은 제외합니다.
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE-2 점수를 계산합니다.
    rouge2 = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    return rouge2["rouge2"] # ROUGE-2 점수만 반환합니다.

# 모델을 훈련시키는 함수입니다.
def train(model, optimizer, dataloader):
    model.train() # 모델을 훈련 모드로 설정합니다.
    train_loss = 0.0 # 훈련 손실을 초기화합니다.
    # 데이터로더로부터 배치 단위로 데이터를 가져옵니다.
    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        # 모델에 데이터를 입력하여 출력(outputs)을 얻습니다.
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels # 라벨은 손실 계산에 사용됩니다.
        )
        loss = outputs.loss # 출력에서 손실 값을 추출합니다.
        train_loss += loss.item() # 손실 값을 누적합니다.
        
        optimizer.zero_grad() # 옵티마이저의 기울기를 초기화합니다.
        loss.backward() # 손실에 대한 역전파를 수행하여 기울기를 계산합니다.
        optimizer.step() # 옵티마이저를 사용하여 모델 파라미터를 업데이트합니다.

    train_loss = train_loss / len(dataloader) # 배치당 평균 손실을 계산합니다.
    return train_loss # 평균 훈련 손실을 반환합니다.

# 모델을 평가하는 함수입니다.
def evaluation(model, dataloader):
    with torch.no_grad(): # 기울기 계산을 비활성화합니다.
        model.eval() # 모델을 평가 모드로 설정합니다.
        val_loss, val_rouge = 0.0, 0.0 # 검증 손실과 ROUGE 점수를 초기화합니다.
        # 데이터로더로부터 배치 단위로 데이터를 가져옵니다.
        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Evaluation"):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits # 출력에서 로짓(logits) 값을 추출합니다.
            loss = outputs.loss # 출력에서 손실 값을 추출합니다.

            logits = logits.detach().cpu().numpy() # 로짓을 CPU로 이동 후 넘파이 배열로 변환합니다.
            label_ids = labels.to("cpu").numpy() # 라벨을 CPU로 이동 후 넘파이 배열로 변환합니다.
            rouge = calc_rouge(logits, label_ids) # ROUGE 점수를 계산합니다.
            
            val_loss += loss.item() # 손실 값을 누적합니다.
            val_rouge += rouge # ROUGE 점수를 누적합니다.

    val_loss = val_loss / len(dataloader) # 배치당 평균 손실을 계산합니다.
    val_rouge = val_rouge / len(dataloader) # 배치당 평균 ROUGE 점수를 계산합니다.
    return val_loss, val_rouge # 평균 손실과 ROUGE 점수를 반환합니다.

# ROUGE 스코어 객체를 로드하고 토크나이저를 전달합니다.
rouge_score = evaluate.load("rouge", tokenizer=tokenizer)
best_loss = 10000 # 최저 손실을 저장하기 위한 변수를 큰 값으로 초기화합니다.

for epoch in range(epochs): # 지정된 에포크 수만큼 반복합니다.
    print(f"--- Epoch {epoch + 1}/{epochs} ---") # 에포크 진행 상황을 시각적으로 표시
    train_loss = train(model, optimizer, train_dataloader) # 훈련 함수를 호출합니다.
    val_loss, val_accuracy = evaluation(model, valid_dataloader) # 평가 함수를 호출합니다.
    # 에포크별 훈련 손실, 검증 손실, ROUGE 점수를 출력합니다.
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Rouge {val_accuracy:.4f}")
    # 현재 검증 손실이 기존의 최저 손실보다 낮으면 모델을 저장합니다.
    if val_loss < best_loss:
        best_loss = val_loss
        # 모델의 가중치를 저장합니다.
        torch.save(model.state_dict(), "../models/BartForConditionalGeneration.pt")
        print("Saved the model weights")

Epoch 1: Train Loss: 2.1637 Val Loss: 1.8464 Val Rouge 0.2535
Saved the model weights
Epoch 2: Train Loss: 1.6029 Val Loss: 1.8931 Val Rouge 0.2596
Epoch 3: Train Loss: 1.2412 Val Loss: 2.0054 Val Rouge 0.2500
Epoch 4: Train Loss: 0.9593 Val Loss: 2.1244 Val Rouge 0.2415
Epoch 5: Train Loss: 0.8476 Val Loss: 2.2706 Val Rouge 0.2347


In [9]:
# 저장된 모델 가중치를 불러와 테스트 데이터셋으로 성능 평가
from transformers import BartForConditionalGeneration # BartForConditionalGeneration 클래스를 임포트합니다.

# BartForConditionalGeneration 모델을 불러옵니다.
model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base").to(device)
# 'facebook/bart-base' 모델을 로드하고, 모델을 지정된 device(CPU 또는 GPU)로 이동시킵니다.

# 저장된 모델의 가중치(state_dict)를 불러와 현재 모델에 적용합니다.
# torch.load() 함수를 이용해 파일 경로에 있는 가중치 파일을 불러옵니다.
# model.load_state_dict() 메서드를 사용하여 불러온 가중치를 모델에 로드합니다.
model.load_state_dict(torch.load("../models/BartForConditionalGeneration.pt"))

# evaluation 함수를 사용하여 모델의 테스트 손실과 ROUGE-2 점수를 계산합니다.
test_loss, test_rouge_score = evaluation(model, test_dataloader)

# 계산된 테스트 손실과 테스트 ROUGE-2 점수를 출력합니다.
print(f"Test Loss : {test_loss:.4f}")
print(f"Test ROUGE-2 Score : {test_rouge_score:.4f}")

Test Loss : 1.8073
Test ROUGE-2 Score : 0.2640
